# Imports

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from datetime import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from time import time
from time import sleep
import pandas as pd
from random import randint
import requests
from bs4 import BeautifulSoup
import re
import math
import numpy as np

# Configs

In [2]:
# login info
# you need to login to Indeed First
# you should change your email and password here 
my_mail = 'sc310542@gmail.com'
my_password = 'p@ssword'

In [77]:
# This is where you add your company dictionary
# it should be in a dict
companys={'tsmc':['台積電'],'media_tech':['聯發科'],'中央研究院':[],'ASUS':[]}

# Functions

In [52]:
# get reviews
#######################3
# ['職位','狀態','標題','總分','勞逸平衡','薪酬/福利','工作保障/升遷制度','管理層','工作文化','評論','優點','缺點'])
########################

def get_reviews(soup) -> list:
    
    tmp_lst =[]
    main_data = soup.find_all('div', {'class':'css-t3vkys eu4oa1w0'})
    
    for data in main_data:
        record={}


        try:
            #
            record['職位'] = data.find("a",attrs={'class':"css-y7yavv emf9s7v0"}).get_text(strip=True)
    #         狀態=data.find("!-- --",attrs={}).get_text(strip=True)
            record['總分'] = data.find("button",attrs={"class":"css-1c33izo e1wnkr790"}).get_text(strip=True)

            # get reviews
            reviews = data.find_all("span",attrs={"class":"css-82l4gy eu4oa1w0"})
            if len(reviews)==3:
                record['標題'] = reviews[0].get_text()
                record['評論'] = reviews[1].get_text()
                record['優點'] = ''
                record['缺點'] = ''
            else:
                record['標題'] = reviews[0].get_text()
                record['評論'] = reviews[1].get_text()
                record['優點'] = reviews[3].get_text()
                record['缺點'] = reviews[4].get_text()


            # get other scores
            elements = data.find_all("span",attrs={"class":"css-1pftocx e1wnkr790"})
            scores = data.find_all("div",attrs={"role":"img"})
            for i in range(len(elements)):
                record[elements[i].get_text()] = scores[i]['aria-label'].split(" ")[0].split('.')[0]         

        except:
            pass

        tmp_lst.append(record)
    return tmp_lst


In [73]:

def get_csv(company_name):
    
    ###################### GET TO PAGE #################################
    # go to company review page
    # avoid obscuring
    webElement = driver.find_element_by_xpath('//a[@href = "https://tw.indeed.com/companies"]' )
    driver.execute_script('arguments[0].click()', webElement)
    driver.forward()
    sleep(1)

    # enter the company name
    search_bar = driver.find_element_by_id("ifl-InputFormField-3")
    search_bar.send_keys(company_name)
    driver.find_element_by_xpath('//button[@type = "submit"]' ).click() 
    driver.forward()
    sleep(1)

    # go to company page
    driver.find_element_by_xpath('//div[@class="css-7lboi8 e1wnkr790"]').click()
    driver.forward()
    sleep(1) 

    # go to company review page
    driver.find_element_by_xpath('//div[@class="css-r228jg eu4oa1w0"]').click()
    driver.forward()
    sleep(1) 

    # get to all review page
    # 有可能沒有所有評論
    try:
        driver.find_element_by_xpath('//a[@class= "css-hvq95c emf9s7v0"]').click()
        driver.forward()
    except:
        pass

    ###################### REVIEW SECTION #################################
    # get reviews
    lst =[]
    try:
        while driver.find_element_by_xpath('//a[@title= "下一個"]'):
            soup = BeautifulSoup(driver.page_source, 'lxml')
            lst += get_reviews(soup)
            driver.find_element_by_xpath('//a[@title= "下一個"]').click()
            driver.forward()
            sleep(5)
    except:
        soup = BeautifulSoup(driver.page_source, 'lxml')
        lst += get_reviews(soup)

    ##################### SAVE TO FILE #################################
    df = pd.DataFrame(lst)
    df.to_csv('./Data/Company_reviews/'+str(company_name)+'.csv')
    
    return len(df)

# Auto Login

In [15]:
# open a search website
opts = webdriver.FirefoxOptions()
opts.headless = True
profile = webdriver.FirefoxProfile()
driver = webdriver.Firefox()

# the login page
driver.get("https://www.indeed.com/account/login")


# use email to login instead of google or other websites 
email = driver.find_element_by_id("ifl-InputFormField-3")
email.send_keys(my_mail)
driver.find_element_by_xpath('//button[@type = "submit"]' ).click() 
sleep(1)
# next_page = driver.current_url
driver.forward()
# driver = driver.get(next_page)
driver.find_element_by_id("auth-page-google-password-fallback").click()
sleep(1)
driver.forward()
password = driver.find_element_by_id("ifl-InputFormField-101")
password.send_keys(my_password)
driver.find_element_by_xpath('//button[@type = "submit"]' ).click() 
driver.forward()
sleep(5)


# 搜尋企業

## A dictionary containing company names

In [78]:
# get company reviews
for company_name in companys: 
    
    # you can remove the print if you like
    
    cnt =0
    print(f'Retrieving.... {company_name}\'s comments')
    cnt = get_csv(company_name)
    print(f'Finish ! Got {cnt} reviews')
    

Retrieving.... tsmc's comments
Finish ! Got 54 reviews
Retrieving.... media_tech's comments
Finish ! Got 12 reviews
Retrieving.... 中央研究院's comments
Finish ! Got 14 reviews
Retrieving.... ASUS's comments
Finish ! Got 109 reviews


## Just getting a specific company's comments

In [79]:
# add your company name here
company_name = '統一企業'

if len(company_name)>0:
    print(f'Retrieving.... {company_name}\'s comments')
    cnt = get_csv(company_name)
    print(f'Finish ! Got {cnt} reviews')
else:
    print(f'Re-enter the company\'s name plzzzzzz')



Retrieving.... 統一企業's comments
Finish ! Got 16 reviews
